### SageMaker self supervised prediction

In [1]:
import sagemaker
import boto3

sagemaker_session = sagemaker.Session()
account_id =  boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name


#role = sagemaker.get_execution_role()
role="arn:aws:iam::{}:role/service-role/AmazonSageMaker-ExecutionRole-20190118T115449".format(account_id)


In [2]:
version_tag="202209180127"
pytorch_custom_image_name=f"large-scale-ptm-ppi:gpu-{version_tag}"
instance_type = "ml.g4dn.2xlarge"  #ml.g4dn.2xlarge
instance_count = 3



In [3]:
docker_repo = "{}.dkr.ecr.{}.amazonaws.com/{}".format(account_id, region, pytorch_custom_image_name)

In [4]:
bucket = "aegovan-data"

In [5]:

abstract_trainfile = "s3://{}/self-supervised/train.json".format(bucket)
abstract_testfile= "s3://{}/self-supervised/test.json".format(bucket)
abstract_valfile="s3://{}/self-supervised/val.json".format(bucket)

abstract_largescale = "s3://{}/chemprotlargescale/input/data_2022080620".format(bucket)

eval_file = abstract_largescale

In [6]:
import datetime
date_fmt = datetime.datetime.today().strftime("%Y%m%d%H")

In [7]:
training_job="selfsupervised-bert-f1-2022-09-17-23-42-42-470"

In [8]:
s3_model_path = f"s3://aegovan-data/selfsupervised_results/{training_job}/output/model.tar.gz"


### Run  prediction

In [9]:
#s3_output_predictions = "s3://aegovan-data/pubmed_asbtract/predictions_largescale_{}_{}/".format(job_prefix,date_fmt)
s3_output_predictions = "s3://aegovan-data/selfsupervised_chemprot/predictions_{}/{}".format(training_job,date_fmt)
s3_input_data = eval_file
s3_data_type="S3Prefix"
usefilter=0
filter_threshold_std=1.0

s3_input_models = s3_model_path
s3_input_vocab = "s3://{}/embeddings/bert/".format(bucket)

In [10]:
s3_input_data, s3_data_type

('s3://aegovan-data/chemprotlargescale/input/data_2022080620', 'S3Prefix')

In [11]:
from sagemaker.network import NetworkConfig
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.processing import ScriptProcessor

script_processor = ScriptProcessor(image_uri=docker_repo,
                                       command=["python"],
                                       env={'mode': 'python', 'PYTHONPATH':'/opt/ml/code'},
                                       role=role,
                                       instance_type=instance_type,
                                       instance_count=instance_count,
                                       max_runtime_in_seconds= 5 * 24 * 60 * 60,
                                       volume_size_in_gb = 200,
                                       network_config=NetworkConfig(enable_network_isolation=False),
                                       base_job_name ="selfsupervised-inference"
                                       )


sm_local_input_models = "/opt/ml/processing/input/data/models"
sm_local_input_data = "/opt/ml/processing/input/data/jsondata"
sm_local_input_vocab = "/opt/ml/processing/input/data/vocab"


sm_local_output = "/opt/ml/processing/output"

input_file_name = s3_input_data.split("/")[-1]

script_processor.run(
        code='../src/inference/chemprot_selfsupervised_batch_predict.py',

        arguments=[
            sm_local_input_data,
            sm_local_input_models,
            sm_local_output,
            "--ensemble", "0",
            "--tokenisor_data_dir", sm_local_input_vocab,           
            "--filter", str(usefilter),
            "--batch", "32",
            "--filterstdthreshold", str(filter_threshold_std)
        ],

        inputs=[
                ProcessingInput(
                    source=s3_input_data,
                    s3_data_type = s3_data_type,
                    destination=sm_local_input_data,
                    s3_data_distribution_type="ShardedByS3Key"),

            ProcessingInput(
                    source=s3_input_models,
                    destination=sm_local_input_models,
                    s3_data_distribution_type="FullyReplicated"),
            
            ProcessingInput(
                    source=s3_input_vocab,
                    destination=sm_local_input_vocab,
                    s3_data_distribution_type="FullyReplicated")
            ],


        outputs=[ProcessingOutput(
                source=sm_local_output, 
                destination=s3_output_predictions,
                output_name='predictions')]
    )





Job Name:  selfsupervised-inference-2022-09-18-02-08-50-050
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://aegovan-data/chemprotlargescale/input/data_2022080620', 'LocalPath': '/opt/ml/processing/input/data/jsondata', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'ShardedByS3Key', 'S3CompressionType': 'None'}}, {'InputName': 'input-2', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://aegovan-data/selfsupervised_results/selfsupervised-bert-f1-2022-09-17-23-42-42-470/output/model.tar.gz', 'LocalPath': '/opt/ml/processing/input/data/models', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'input-3', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://aegovan-data/embeddings/bert/', 'LocalPath': '/opt/ml/processing/input/data/vocab', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3Compression

2022-09-18 02:14:35,405 - models.bert_model_factory - INFO - Retrieving key model_fine_tune with default 0, found 0
2022-09-18 02:14:35,405 - models.bert_model_factory - INFO - Retrieving model complete
2022-09-18 02:14:35,405 - dataset_builder - INFO - Retrieving Tokeniser
2022-09-18 02:14:35,405 - models.bert_model_factory - INFO - Retrieving Tokeniser
2022-09-18 02:14:35,405 - models.bert_model_factory - INFO - Retrieving key tokenisor_max_seq_len with default 512, found 512
2022-09-18 02:14:35,405 - models.bert_model_factory - INFO - Retrieving key tokenisor_lower_case with default 0, found 0
2022-09-18 02:14:35,405 - models.bert_model_factory - INFO - Retrieving key pretrained_model with default bert-base-cased, found /opt/ml/input/data/PRETRAINED_MODEL
2022-09-18 02:14:35,405 - models.bert_model_factory - INFO - Retrieving key tokenisor_data_dir with default /opt/ml/input/data/PRETRAINED_MODEL, found /opt/ml/processing/input/data/vocab
2022-09-18 02:14:35,683 - models.bert_model_

/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,
2022-09-18 03:05:52,172 - inference.ensemble_predictor - INFO - Using devices ['cuda:0']
2022-09-18 03:05:52,283 - inference.predictor - INFO - Using device cuda:0
2022-09-18 03:05:52,657 - inference.batch_predict - INFO - Records to write: 91621
2022-09-18 03:05:52,657 - inference.batch_predict - INFO - Writing to file /opt/ml/processing/output/pubmed19n0649.json.rel.json.json
2022-09-18 03:05:55,052 - inference.batch_predict - INFO - Completed file /opt/ml/processing/input/data/jsondata/pubmed19n0649.json.rel.json
2022-09-18 03:05:55,151 - inference.batch_predict - INFO - Running inference on file /opt/ml/processing/input/data/jsondata/pubmed19n0616.json.rel.json with output in /opt/ml/processi

2022-09-18 03:59:26,604 - inference.ensemble_predictor - INFO - Using devices ['cuda:0']
2022-09-18 03:59:26,715 - inference.predictor - INFO - Using device cuda:0
2022-09-18 04:00:20,014 - inference.predictor - INFO - Completed inference cuda:0
2022-09-18 04:00:20,017 - inference.ensemble_predictor - INFO - Computing average 
2022-09-18 04:00:20,048 - inference.ensemble_predictor - INFO - Computing ensemble prediction 
2022-09-18 04:00:20,265 - inference.ensemble_predictor - INFO - Completed ensemble prediction 
/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,
2022-09-18 04:00:29,632 - inference.batch_predict - INFO - Records to write: 98385
2022-09-18 04:00:29,632 - inference.batch_predict - INFO - Writing to file /opt/ml/processing/output/pu

2022-09-18 04:50:35,924 - inference.predictor - INFO - Completed inference cuda:0
2022-09-18 04:50:35,927 - inference.ensemble_predictor - INFO - Computing average 
2022-09-18 04:50:35,954 - inference.ensemble_predictor - INFO - Computing ensemble prediction 
2022-09-18 04:50:36,164 - inference.ensemble_predictor - INFO - Completed ensemble prediction 
/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,
2022-09-18 04:50:45,041 - inference.batch_predict - INFO - Records to write: 93664
2022-09-18 04:50:45,041 - inference.batch_predict - INFO - Writing to file /opt/ml/processing/output/pubmed19n0662.json.rel.json.json
2022-09-18 04:50:47,510 - inference.batch_predict - INFO - Completed file /opt/ml/processing/input/data/jsondata/pubmed19n0662.json.r

...

2022-09-18 20:00:22,177 - inference.predictor - INFO - Completed inference cuda:0
2022-09-18 20:00:22,180 - inference.ensemble_predictor - INFO - Computing average 
2022-09-18 20:00:22,213 - inference.ensemble_predictor - INFO - Computing ensemble prediction 
2022-09-18 20:00:22,461 - inference.ensemble_predictor - INFO - Completed ensemble prediction 
/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,
2022-09-18 20:00:33,251 - inference.batch_predict - INFO - Records to write: 111266
2022-09-18 20:00:33,251 - inference.batch_predict - INFO - Writing to file /opt/ml/processing/output/pubmed19n0628.json.rel.json.json
2022-09-18 20:00:36,179 - inference.batch_predict - INFO - Completed file /opt/ml/processing/input/data/jsondata/pubmed19n0628.json.

2022-09-18 20:30:53,982 - inference.predictor - INFO - Completed inference cuda:0
2022-09-18 20:30:53,983 - inference.ensemble_predictor - INFO - Computing average 
2022-09-18 20:30:53,988 - inference.ensemble_predictor - INFO - Computing ensemble prediction 
2022-09-18 20:30:54,030 - inference.ensemble_predictor - INFO - Completed ensemble prediction 
/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,
2022-09-18 20:30:55,650 - inference.batch_predict - INFO - Records to write: 18178
2022-09-18 20:30:55,650 - inference.batch_predict - INFO - Writing to file /opt/ml/processing/output/pubmed19n0669.json.rel.json.json
2022-09-18 20:30:56,154 - inference.batch_predict - INFO - Completed file /opt/ml/processing/input/data/jsondata/pubmed19n0669.json.r

2022-09-18 21:05:45,699 - inference.ensemble_predictor - INFO - Using devices ['cuda:0']
2022-09-18 21:05:45,808 - inference.predictor - INFO - Using device cuda:0
2022-09-18 21:24:59,629 - inference.predictor - INFO - Completed inference cuda:0
2022-09-18 21:24:59,632 - inference.ensemble_predictor - INFO - Computing average 
2022-09-18 21:24:59,660 - inference.ensemble_predictor - INFO - Computing ensemble prediction 
2022-09-18 21:24:59,877 - inference.ensemble_predictor - INFO - Completed ensemble prediction 
/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,
2022-09-18 21:25:09,307 - inference.batch_predict - INFO - Records to write: 96957
2022-09-18 21:25:09,308 - inference.batch_predict - INFO - Writing to file /opt/ml/processing/output/pu

2022-09-18 21:56:18,889 - inference.batch_predict - INFO - Completed file /opt/ml/processing/input/data/jsondata/pubmed19n0630.json.rel.json
2022-09-18 21:56:18,950 - inference.batch_predict - INFO - Running inference on file /opt/ml/processing/input/data/jsondata/pubmed19n0621.json.rel.json with output in /opt/ml/processing/output/pubmed19n0621.json.rel.json.json
2022-09-18 21:56:18,952 - inference.batch_predict - INFO - Processing data file /opt/ml/processing/input/data/jsondata/pubmed19n0621.json.rel.json
2022-09-18 21:56:18,957 - inference.batch_predict - INFO - Using args :{'datasetfactory': 'datasets.chemprot_selfsupervised_dataset_factory.ChemprotSelfsupervisedDatasetFactory', 'traindir': '/opt/ml/input/data/train', 'valdir': '/opt/ml/input/data/val', 'testdir': None, 'modelfactory': 'models.bert_model_factory.BertModelFactory', 'pretrained_model_dir': '/opt/ml/input/data/PRETRAINED_MODEL', 'kfoldtrainprefix': None, 'outdir': '/opt/ml/output/data', 'modeldir': '/opt/ml/model', '

KeyboardInterrupt: 